In [2]:
# -*- coding: utf8 -*-
import pandas as pd

In [3]:
df = pd.read_excel("done.xlsx")

In [4]:
df.head()

,Unnamed: 0,college,department,dcode,name,test id,scode,n1,n2
0,0,國立臺灣大學,中國文學系,1,傅安沛,22082041,1,41,41
1,1,國立臺灣大學,中國文學系,1,杜芊曄,22035237,1,41,41
2,2,國立臺灣大學,中國文學系,1,李姿瑩,22051727,1,41,41
3,3,國立臺灣大學,中國文學系,1,陳怡親,22094937,1,41,41
4,4,國立臺灣大學,中國文學系,1,江仕達,22088623,1,41,41


In [5]:
# a-1. 錄取多少學生
print(len(df['test id']))

88936


In [6]:
# a-2. 學校數量
print(len(df['college'].unique()))

71


In [8]:
df_1 = pd.read_excel("collegename_clean.xlsx")
df_1.head()

,college,department,dcode,scode,n1,n2
0,國立臺灣大學,中國文學系,1,1,41,41
1,國立臺灣大學,外國語文學系,2,1,73,73
2,國立臺灣大學,歷史學系,3,1,36,36
3,國立臺灣大學,哲學系,4,1,36,36
4,國立臺灣大學,人類學系,5,1,32,32


In [9]:
# a-3. 錄取人數最多的前十間學校

college_list = df_1['college'].unique()
number_dict = {} # 每間學校 + 總錄取人數

for i in range(len(college_list)):
    student_num = 0
    for j in range(len(df['college'])):
        if df['college'][j] == college_list[i]:
            student_num += 1
        else:
            pass
    number_dict[college_list[i]] = student_num

# print(number_dict)

def order_dict(dicts, n):
    result = []
    result1 = []
    result_num = []
    p = sorted([(k, v) for k, v in dicts.items()], reverse=True)
    s = set()
    
    for i in p:
        s.add(i[1])
        
    for i in sorted(s, reverse=True)[:n]:
        for j in p:
            if j[1] == i:
                result.append(j)
    
    for r in result:
        school_num = r[0] + str(r[1])
        result1.append(school_num)
    
    return result1

print(order_dict(number_dict, 10))

['中國文化大學4292', '淡江大學3581', '輔仁大學3393', '銘傳大學3193', '逢甲大學2933', '國立臺灣大學2861', '東海大學2806', '真理大學2424', '東吳大學2398', '實踐大學2376']


In [10]:
# c. 由學校名稱將錄取學校分為公立和私立，公私立學校分別錄取多少學生

num_df = pd.DataFrame([number_dict.keys(), number_dict.values()]).T # 轉 dataframe
num_df.columns = ['college', 'student num']
num_df.rename(columns={0: 'Stud。ent Name'}, inplace=True)

# print(num_df['college'][0][:1])

public_num = 0
private_num = 0

for i in range(len(num_df['college'])):
    if num_df['college'][i][0][:1] == "國":
        public_num += num_df['student num'][i]
    else:
        private_num += num_df['student num'][i]

print(public_num)
print(private_num)

28297
60639


In [24]:
# 取出考生姓名中的「主名」與「次名」，例如林怡君的主名為「怡」、次名為「君」、
#「名字」為「怡君」。公立大學中出現次數最多的名字為何？私立大學呢？

public_name = {}
private_name = {}

# print(df['name'][0][-2:])

for i in range(len(df['name'])):
    if df['college'][i][0] == "國":
        temp_name = df['name'][i][-2:]
        if temp_name not in public_name.keys():
            public_name[temp_name] = 1
        else:
            public_name[temp_name] = public_name.get(temp_name) + 1
    
    else:
        temp_name = df['name'][i][-2:]
        if temp_name not in private_name.keys():
            private_name[temp_name] = 1
        else:
            private_name[temp_name] = private_name.get(temp_name) + 1

In [27]:
import operator

max_public = max(public_name, key = public_name.get)
max_private = max(private_name, key = private_name.get)

print('max_public ', max_public)
print('max_private ', max_private)

max_public  怡君
max_private  雅婷


In [28]:
# 如果我們將榜單中只出現一次的名字稱為「獨特名字」，公私立大學錄取學生的名字為「獨特名字」的比例各為多少? 何者較高? 是否有顯著差異?

special_pub_df = pd.DataFrame([public_name.keys(), public_name.values()]).T # 轉 dataframe
special_pub_df.columns = ['name', 'count']

special_pri_df = pd.DataFrame([private_name.keys(), private_name.values()]).T # 轉 dataframe
special_pri_df.columns = ['name', 'count']

In [34]:
spec_pub_name = 0

for i in range(len(special_pub_df['name'])): # public
    if special_pub_df['count'][i] == 1:
        spec_pub_name += 1
    else:
        pass

spec_pri_name = 0
for i in range(len(special_pri_df['name'])): # private
    if special_pri_df['count'][i] == 1:
        spec_pri_name += 1
    else:
        pass

print('spec_pub_name ', spec_pub_name / public_num) # public_num 公立錄取人數
print('spec_pri_name ', spec_pri_name / private_num) # private_num 私立錄取人數

spec_pub_name  0.5201258083895819
spec_pri_name  0.3962136578769439


In [30]:
special_pri_df.head()

,name,count
0,愷毅,1
1,曉晴,3
2,聖傑,21
3,翔阡,1
4,偉勝,5


In [45]:
# 是否有顯著差異
from scipy import stats
import numpy as np

def ttest(group1, group2):
    mean1 = np.mean(group1)
    mean2 = np.mean(group2)
    std1 = np.std(group1)
    std2 = np.std(group2)
    nobs1 = len(group1)
    nobs2 = len(group2)
    
    modified_std1 = np.sqrt(np.float32(nobs1)/
                    np.float32(nobs1-1)) * std1
    modified_std2 = np.sqrt(np.float32(nobs2)/
                    np.float32(nobs2-1)) * std2
    statistic, pvalue = stats.ttest_ind_from_stats( 
               mean1=mean1, std1=modified_std1, nobs1=nobs1,   
               mean2=mean2, std2=modified_std2, nobs2=nobs2 )
    return statistic, pvalue

print(ttest(special_pub_df['count'], special_pri_df['count']))

(-12.208802782616774, 3.1140275515111545e-34)


In [50]:
# 公私立大學中出現次數最多的前十個名字、出現次數、各個名字所佔錄取學生的比例、以及常見名字的累積比例

special_pub_10_df = special_pub_df.sort_values(['count'],ascending=False)
special_pub_10_df.head(10)

,name,count
744,怡君,73
550,雅婷,72
7,宗翰,59
178,家豪,44
710,怡婷,40
16,雅雯,38
692,佩珊,35
2637,婉婷,35
1110,欣怡,35
734,佳蓉,30


In [51]:
special_pri_10_df = special_pri_df.sort_values(['count'],ascending=False)
special_pri_10_df.head(10)

,name,count
118,雅婷,218
288,怡君,181
149,欣怡,101
1496,怡婷,100
325,佳蓉,99
601,家豪,94
1611,宗翰,93
201,婉婷,85
644,雅雯,77
183,佩珊,75


In [60]:
special_pub_10_df.insert(2, "ratio", '')
special_pri_10_df.insert(2, "ratio", '')

ValueError: cannot insert ratio, already exists

In [99]:
special_pub_10_df['ratio'] = list(map(lambda x: x/public_num, special_pub_10_df['count'])) # public_num 公立錄取人數 28297
special_pub_10_df.head(10)

,name,count,ratio
744,怡君,73,0.002580
550,雅婷,72,0.002544
7,宗翰,59,0.002085
178,家豪,44,0.001555
710,怡婷,40,0.001414
16,雅雯,38,0.001343
692,佩珊,35,0.001237
2637,婉婷,35,0.001237
1110,欣怡,35,0.001237
734,佳蓉,30,0.001060


In [100]:
special_pri_10_df['ratio'] = list(map(lambda x: x/private_num, special_pri_10_df['count'])) # private_num 私立錄取人數 60639
special_pri_10_df.head(10)

,name,count,ratio
118,雅婷,218,0.003595
288,怡君,181,0.002985
149,欣怡,101,0.001666
1496,怡婷,100,0.001649
325,佳蓉,99,0.001633
601,家豪,94,0.001550
1611,宗翰,93,0.001534
201,婉婷,85,0.001402
644,雅雯,77,0.001270
183,佩珊,75,0.001237
